In [3]:
#Import necessary libraries
import numpy as np #This works with numbers
import pandas as pd #This allows us to work with a dataset
import matplotlib.pyplot as plt #This allows us to plot data
from sklearn.model_selection import train_test_split #For splitting data for training and test
from sklearn.preprocessing import StandardScaler #For scaling features
from sklearn import datasets, svm 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score #For metrics and evaluation algorithm
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier #For KNN
from sklearn.preprocessing import KBinsDiscretizer

In [4]:
#load data into dataframe
data = pd.read_csv('HCV-Egy-Data.csv')

# Remove Outliers 
X = data
col = list(X.drop('Baselinehistological staging',axis=1).columns)
for i in col:
    y = data[i]
    removed_outliers = y.between(y.quantile(.02), y.quantile(.98))
    index_names = data[~removed_outliers].index 
    data.drop(index_names, inplace=True)
    
#dropping histoligical baseline due to duplicate representation of data
data = data.drop('Baseline histological Grading', axis = 1)
    
#seperate X and y
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

#add a FIB-4
fib4 = (X['Age']*X['AST1'])/(X['Plat']*np.sqrt(X['ALT1']))
modapri = (X['Age']*X['AST1'])/(X['Plat']*100)
liverDam = X['AST1']/X['ALT1']

X['fib4']=fib4
X['modapri']=modapri

ModX = X.iloc[:,0:3]


ModX['ALT1']=X['ALT1']
ModX['ALT4']=X['ALT4']
#ModX['ALT 12']=X['ALT 12']
ModX['ALT 24']=X['ALT 24']
ModX['ALT 48']=X['ALT 48']

#ModX['ALT after 24 w']=X['ALT after 24 w']
ModX['AST1']=X['AST1']

#ModX['RNA Base']=X['RNA Base']
#ModX['RNA 4']=X['RNA 4']
#ModX['RNA 12']=X['RNA 12']
#ModX['RNA EOT']=X['RNA EOT']

#ModX['Jaundice']=X['Jaundice']

ModX
ModX['fib4'] = fib4
ModX['modapri'] = modapri
ModX['liverDam'] = liverDam

ModX

,Age,Gender,BMI,ALT1,ALT4,ALT 24,ALT 48,AST1,fib4,modapri,liverDam
3,49,2,33,64,109,88,77,43,0.001798,0.000144,0.671875
5,58,2,22,104,121,65,114,66,0.002860,0.000292,0.634615
6,42,2,26,57,113,107,80,78,0.002448,0.000185,1.368421
7,48,2,30,112,80,45,53,119,0.002497,0.000264,1.062500
9,45,1,30,68,72,81,43,55,0.003056,0.000252,0.808824
...,...,...,...,...,...,...,...,...,...,...,...
1369,32,1,26,98,102,86,110,77,0.001777,0.000176,0.785714
1373,46,1,27,80,64,92,79,118,0.005642,0.000505,1.475000
1376,46,1,27,49,77,103,120,40,0.002176,0.000152,0.816327
1377,52,2,25,50,98,50,122,109,0.004379,0.000310,2.180000


In [5]:
#Split data into training and test data
X_train, X_test, y_train, y_test = train_test_split(ModX, y, test_size = 0.20, random_state = 0)

#Feature scale so that one feature doesn't have more influence than another
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
#Here we are going to make a simple SVM classifier
classifier = svm.NuSVC(nu = 0.4, kernel='rbf')
#Carry out cross validation 10 fold and find the mean of the score to get overall accuracy for train data
scores = cross_val_score(classifier, X_train, y_train, cv=10)
print(scores)
print(scores.mean())

[0.26086957 0.26086957 0.15942029 0.2173913  0.23529412 0.29411765
 0.22058824 0.26470588 0.19117647 0.25      ]
0.23544330775788574


In [12]:
#train the classifier on train set and then print out its overall accuracy
classifier.fit(X_train, y_train)

print(classifier.score(X_train,y_train))

0.9312865497076024


In [13]:
#Determine the overall accuracy score of the test data set
print(classifier.score(X_test,y_test))

0.36257309941520466


In [15]:
#Making predictions
y_pred = classifier.predict(X_test)
#To evaluate algorithm we will print the confusion matrix and other metrics
print("The confusion matrix is:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

The confusion matrix is:
[[19  8  9 11]
 [10  7  8  7]
 [11  7 22 12]
 [12 11  3 14]]
              precision    recall  f1-score   support

           1       0.37      0.40      0.38        47
           2       0.21      0.22      0.22        32
           3       0.52      0.42      0.47        52
           4       0.32      0.35      0.33        40

    accuracy                           0.36       171
   macro avg       0.35      0.35      0.35       171
weighted avg       0.37      0.36      0.37       171



In [11]:
#Training with k-nearest neighbor (KNN)
classifier2 = KNeighborsClassifier(n_neighbors = 50)
classifier2.fit(X_train, y_train)
#Making predictions
y_pred = classifier2.predict(X_test)
#To evaluate algorithm we will print the confusion matrix and other metrics
print("The confusion matrix is:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

The confusion matrix is:
[[ 8  7  7 25]
 [ 5  7  6 14]
 [10 15  7 20]
 [10  6 11 13]]
              precision    recall  f1-score   support

           1       0.24      0.17      0.20        47
           2       0.20      0.22      0.21        32
           3       0.23      0.13      0.17        52
           4       0.18      0.33      0.23        40

    accuracy                           0.20       171
   macro avg       0.21      0.21      0.20       171
weighted avg       0.21      0.20      0.20       171

